In [5]:
from __future__ import print_function

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import os
import functools
import operator
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.stem import 	WordNetLemmatizer
from num2words import num2words

from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import LatentDirichletAllocation

pd.set_option('precision', 2)


[nltk_data] Downloading package stopwords to /Users/vm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Import dataset

To import the dataset. For now, import using from a csv file. In a later iteration, provide an option to import files. [ To add: An option to read from webpages, different separators]

In the example, we are looking at a small collection of short senyences related to two different topics. This should allow us to envision the effects of changes in method.

In [6]:
# example = pd.read_csv('/Users/vm/OneDrive/UNC/lsi/text_corpus.txt', header=None, names=['TEXT'],sep = "\n")
# example = example['TEXT'].tolist()

example = ['Human machine interface for ABC computer applications',
           'A survey of user opinion of computer system response time',
           'The EPS user interface management system',
           'System and human system engineering testing of EPS',
           'Relation of user perceived response time to error measurement',
           'The generation of random, binary, ordered trees',
           'The intersection graph of paths in trees',
           'Graph minor IV: Widths of trees and well-quasi-ordering',
           'Graph minor: A survey']



In [7]:
#Some Functions
def flatten(t):
    return [item for sublist in t for item in sublist]
def count_tokens(w,example_tok):
    count = 0
    for sent in example_tok:
        if w in sent:
            count+= 1
    return count

# Preprocessing
Before constructing the matrix, the text should be preprocessed. This involved 
1) retaining the stem of the words by removing plurals, tenses
2) converting numbers to words
3) removing stop words and special characters

In [10]:
## Vectorize
#lower case
example_LC = np.char.lower(example)

#print(example_LC)


# Stem
example_stem = []
#different stemmer algorithms may be used
ps = PorterStemmer()
for sentence in example_LC:
  text = ' '.join([ps.stem(word) for word in sentence.split()])
  example_stem.append(text)

print('stemmed')


# Lemmatize
example_lem = []
#different stemmer algorithms may be used
lm = WordNetLemmatizer()
for sentence in example_LC:
  text = ' '.join([lm.lemmatize(word) for word in sentence.split()])
  example_lem.append(text)

print('lemmatized')


# Number to words
example_num = []
for sentence in example_lem:
  text = ' '.join([num2words(word) if word.isnumeric() else word for word in sentence.split()])
  example_num.append(text)
print('number to worded')
    

#remove stopwords
stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)
example_stop = []
for sentence in example_num:
  text = ' '.join([word for word in sentence.split() if word not in stopwords_dict])
  example_stop.append(text)
print('stopwords removed')


# remove punctuations
symbols = "!\"#$%&()*+-.,/:;<=>?@[\]^_`{|}~\n"
for i in symbols:
    example_stop = np.char.replace(example_stop, i, ' ')
print('punctuations removed')

# print(example_stop)

example_tok = []
for sentence in example_stop:
  example_tok.append(nltk.word_tokenize(sentence))


print('tokenized')


# Prepare a list of tokens that are repeated atleast once
token_list = flatten(example_tok)
token_list = list(set(token_list))
print(len(token_list))
new_list = []
for word in token_list: 
    if count_tokens(word,example_tok)>1:
        new_list.append(word)
print(new_list)


stemmed
lemmatized
number to worded
stopwords removed
punctuations removed
tokenized
34
['tree', 'interface', 'system', 'graph', 'computer', 'user', 'human', 'survey', 'eps', 'minor', 'time', 'response']


# Constructing the TF-IDF matrix 

The term frequency for each term $tf(w,d)$ is the frequency of the word $w$ in document $d$. Document frequency $df(w)$ is the number of documents the word $w$ appears in. If a word appears in too many documents, it is a common word and does not help with understanding how the documents are different. Therefore, the term frequency matrix is weighted by the inverse of the document frequency.

examples of term-document matrices

Only term frequency
$$T[w,d] = tf[w,d]$$

TF-IDF
$$TD = tf[w,d]*\log(N_t/df[w])$$

Other term document matricies

#A bit more about TF-IDF and Vector space Models
VSMs (1970, Salton) and TF-IDF (1975, Salton et.. al)


In [11]:

############################################

N_d = len(example_tok)
print(N_d)

#termlist =  functools.reduce(operator.iconcat, example_tok, [])
#termlist = list(set(termlist))

N_t = len(new_list)
tf_mat = np.zeros(shape = (N_d,N_t))
tf_mat = csr_matrix(tf_mat)
tf_idf = csr_matrix(tf_mat)

# Document frequency
DF = {}
for i in range(N_d):
    tokens = example_tok[i]
    for w in tokens:
        if w in new_list:
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}


for i in DF:
  DF[i] = len(DF[i])
DF = np.fromiter(DF.values(), dtype=float) 

idf = np.log(N_t/DF) +1



# print(new_list)


#term frequency and tfidf
for i in range(N_d):
    # i = 3
    print(i)
    tokens = example_tok[i]
    #print(tokens)

    words_count = len(example_tok[i])
    counter = Counter(example_tok[i])

    for token in new_list:
        try:
            ind = new_list.index(token)

            tf = counter[token]/words_count
            tf2 = counter[token]

            tf_mat[i,ind] = tf2
            tf_idf[i,ind] = tf*idf[ind]
            #df1 = DF[token]
        except:
            pass

tf_mat_df = pd.DataFrame(tf_mat.toarray(),columns = new_list)
tf_idf_df = pd.DataFrame(tf_idf.toarray(),columns = new_list)
#print(tf_idf_df)
tf_mat = tf_mat.toarray()
from scipy import stats
print(len(new_list))
print(len(token_list))
print(N_d)

9
0
1
2
3
4
5
6
7
8
12
34
9


/Users/vm/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


# LSA

Using sklearn

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vector = tfidf_vectorizer.fit_transform(example)

In [21]:
k =2
U,s,Vt = svds(tf_mat,k=k)
# print(np.transpose(Vt))
print(U)

kmeans = KMeans(n_clusters=k, random_state=0).fit(U)
print(kmeans.labels_)
# plt.scatter(U[:,0],U[:,4],U[:,5],c=kmeans.labels_.astype(float))

tf_hat = np.matmul(np.matmul(U,np.diag(s)),Vt)
print(pd.DataFrame(tf_hat, columns = new_list))
y, rho =stats.spearmanr(tf_hat[:,2], tf_hat[:,4])
print(y)
print(stats.spearmanr(tf_hat[:,2], tf_hat[:,4]))
print(stats.spearmanr(tf_hat[:,2], tf_hat[:,10]))
kmeans = KMeans(n_clusters=k, random_state=0).fit(U)



[[ 0.05591352  0.1973928 ]
 [-0.16559288  0.60599027]
 [ 0.12731206  0.46291751]
 [ 0.23175523  0.54211442]
 [-0.10677472  0.27946911]
 [-0.19284794  0.00381521]
 [-0.43787488  0.01463147]
 [-0.6151219   0.02413684]
 [-0.52993707  0.08195737]]
[0 0 0 0 0 1 1 1 1]
   tree  interface  system  graph  computer  user  human  survey   eps  minor  \
0 -0.06       0.14    0.45  -0.06      0.15  0.26   0.16    0.10  0.22  -0.04   
1  0.23       0.37    1.23   0.34      0.51  0.84   0.40    0.53  0.55   0.25   
2 -0.14       0.33    1.05  -0.15      0.36  0.61   0.38    0.23  0.51  -0.10   
3 -0.27       0.40    1.27  -0.30      0.41  0.70   0.47    0.21  0.63  -0.21   
4  0.14       0.16    0.56   0.20      0.24  0.39   0.18    0.27  0.24   0.15   
5  0.24      -0.03   -0.07   0.31      0.02  0.03  -0.05    0.14 -0.07   0.22   
6  0.55      -0.07   -0.15   0.69      0.06  0.08  -0.12    0.31 -0.14   0.50   
7  0.77      -0.10   -0.21   0.98      0.09  0.12  -0.16    0.44 -0.20   0.71   
8  0.66

# SVD Vectorizer

# SVD Standard 

TF matrix variants
1) standard term frequency. Leads to bias towards documents with many term occurrences.

2) TF sum - The term frequencies are normalized by the total number of terms in the document. Leads to biases towards short documents
$$TF_{\text{sum}} = \frac{tf_{L}[w,d]}{\sum_w{tf_{L}[w,d]}}$$

The biases induced by variants 1 and 2 cannot be observed in the example being considered, since all documents are of equal length and have roughly the same number of terms.

3) TF max - The term frequencies are normalized by the highest number of occrrences for any term in the document. Mitigates the biases of variants 1 and 2
$$TF_{\text{max}} = \frac{tf_{L}[w,d]}{\max_w{tf_{L}[w,d]}}$$

4) TF log - reduces the impact of each additional occurrence
$$TF_{\text{log}} = \log(1+ tf_{L}[w,d])$$

5) TF frac - Here $K_d$ is the pivoted document length. If $L_D = \text{avg}({\ell_d})$, where $\ell_d$ is the length of document $d$, the pivoted document length is given by $K_d = \ell_d/L_D$. It is less than one for short documents and more than one for longer documents. Like the logarithmic approach, the effect of each additional additional occurrence is reduced, the reduction is more pronounced for larger documents


$$TF_{\text{frac,K}} = \frac{tf_{L}[w,d]}{tf_{L}[w,d] + K_d}$$

The graph in the next cell shows the comparison between the standard, log and the fractional version of TF (for $K_d = 0.5,2$)

In [ ]:
## Showing the effect of different 
x = np.array(range(10))
print(np.log(x))
plt.plot(x,x)
plt.plot(x,x/(x+2))
plt.plot(x,x/(x+0.5))
plt.plot(x, np.log(x))
plt.xlabel('Frequency')
plt.ylabel('TF')
plt.legend(['TF std','K_d = 0.5','K_d = 2','log'])

In [32]:
k = 2
U,s,Vt = svds(tf_mat,k=k)
# plt.scatter(U[:,0],U[:,1],c=kmeans.labels_.astype(float))
np.set_printoptions(precision=3)
tf_hat = np.matmul(np.matmul(U,np.diag(s)),Vt)
# print(pd.DataFrame(tf_hat, columns = new_list))
y, rho =stats.spearmanr(tf_hat[:,2], tf_hat[:,4])
print(y)

kmeans = KMeans(n_clusters=k, random_state=0).fit(U)

corr_raw = np.zeros([N_d,N_d])
for i in range(N_d):
    for j in range(i+1):
        t,r = stats.spearmanr(tf_hat[i,:], tf_hat[j,:])
        corr_raw[i][j] = t
corr = np.zeros([N_d,N_d])
for i in range(N_d):
    for j in range(i+1):
        t,r = stats.spearmanr(tf_mat[i,:], tf_mat[j,:])
        corr[i][j] = t
print(corr_raw)
print(tf_mat_df)


0.9166666666666666
[[ 1.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.846  1.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 1.     0.846  1.     0.     0.     0.     0.     0.     0.   ]
 [ 1.     0.846  1.     1.     0.     0.     0.     0.     0.   ]
 [ 0.719  0.972  0.719  0.719  1.     0.     0.     0.     0.   ]
 [-0.839 -0.558 -0.839 -0.839 -0.389  1.     0.     0.     0.   ]
 [-0.839 -0.558 -0.839 -0.839 -0.389  1.     1.     0.     0.   ]
 [-0.839 -0.558 -0.839 -0.839 -0.389  1.     1.     1.     0.   ]
 [-0.804 -0.481 -0.804 -0.804 -0.298  0.979  0.979  0.979  1.   ]]
   tree  interface  system  graph  computer  user  human  survey  eps  minor  \
0   0.0        1.0     0.0    0.0       1.0   0.0    1.0     0.0  0.0    0.0   
1   0.0        0.0     1.0    0.0       1.0   1.0    0.0     1.0  0.0    0.0   
2   0.0        1.0     1.0    0.0       0.0   1.0    0.0     0.0  1.0    0.0   
3   0.0        0.0     2.0    0.0       0.0   0.0    1.0     0.0  

In [31]:
k = 2
U,s,Vt = svds(tf_idf,k=k)
# plt.scatter(U[:,0],U[:,1],c=kmeans.labels_.astype(float))
np.set_printoptions(precision=3)
tf_hat = np.matmul(np.matmul(U,np.diag(s)),Vt)
# print(pd.DataFrame(tf_hat, columns = new_list))
y, rho =stats.spearmanr(tf_hat[:,2], tf_hat[:,4])
print(y)

kmeans = KMeans(n_clusters=k, random_state=0).fit(U)

corr_raw = np.zeros([N_d,N_d])
for i in range(N_d):
    for j in range(i+1):
        t,r = stats.spearmanr(tf_hat[i,:], tf_hat[j,:])
        corr_raw[i][j] = t
corr = np.zeros([N_d,N_d])
for i in range(N_d):
    for j in range(i+1):
        t,r = stats.spearmanr(tf_idf[i,:], tf_idf[j,:])
        corr[i][j] = t
print(corr_raw)
print(tf_idf_df)

0.9166666666666666
[[ 1.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.706  1.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 1.     0.706  1.     0.     0.     0.     0.     0.     0.   ]
 [ 1.     0.706  1.     1.     0.     0.     0.     0.     0.   ]
 [ 0.937  0.874  0.937  0.937  1.     0.     0.     0.     0.   ]
 [-0.944 -0.552 -0.944 -0.944 -0.825  1.     0.     0.     0.   ]
 [-0.944 -0.552 -0.944 -0.944 -0.825  1.     1.     0.     0.   ]
 [-0.944 -0.552 -0.944 -0.944 -0.825  1.     1.     1.     0.   ]
 [-0.881 -0.476 -0.881 -0.881 -0.762  0.951  0.951  0.951  1.   ]]
   tree  interface  system  graph  computer  user  human  survey   eps  minor  \
0  0.00       0.47    0.00   0.00      0.40  0.00   0.47    0.00  0.00    0.0   
1  0.00       0.00    0.40   0.00      0.34  0.34   0.00    0.40  0.00    0.0   
2  0.00       0.56    0.56   0.00      0.00  0.48   0.00    0.00  0.56    0.0   
3  0.00       0.00    0.93   0.00      0.00  0.00   0.47    0.

# LSA paper

In [ ]:
corr_raw = np.zeros([N_d,N_d])
for i in range(N_d):
    for j in range(i+1):
        t,r = stats.spearmanr(tf_hat[i,:], tf_hat[j,:])
        corr_raw[i][j] = t
corr = np.zeros([N_d,N_d])
for i in range(N_d):
    for j in range(i+1):
        t,r = stats.spearmanr(tf_mat[i,:], tf_mat[j,:])
        corr[i][j] = t
print(corr_raw)
print(corr)